# Underdog Betting Principle - Proof of Concept

Here we assume that the underlying assumpions of the underdog betting principle are true.
We then conduct a monte carlo simulation to simulate the earnings over time. This is a proof of concept.

In [36]:
#parameters
#Bookies line: l_f and l_u in [0.5,1], i.e. probability that favorite wins for favorite and underdog betting lines,
# # # # # # # # # # # # # # # # # # # # discrepency comes from bookie's margin m
#Bookie's perceived odds: q in [0.5,1], i.e probability that favorite wins, without margin effect
#Bookie's margin: m, i.e. percent taken from winnings
#Underdog Betting Principle hypothesized odds: p in [0.5,1], i.e. true probability that favorite wins
#Safety Factor: SF = p/q if p>=q, and SF = (1-p)/(1-q) if p<q, for same favorite case
############### SF = p/(1-q), for different favorite case
#p, q, l, m are all known
#Earnings per bet = SF/m --> set SF/m to desired amount, with higher SF resulting in less playable bets 
# # # # # # # # # # # # # # # # # #  but higher earnings per bet -> find the balance for best cumulative earnings

In [32]:
#generating 1 betting scenario
import numpy as np
def generate_scenario(m, max_number_of_bets_per_scenario = 10000, max_amount_bet = 10):
    A = np.random.uniform(1, max_number_of_bets)
    B = np.random.uniform(1, max_number_of_bets)
    A_dollars = 0
    for i in range(A):
        A_dollars += np.random.uniform(1, max_amount_bet)
    for i in range(B):
        B_dollars += np.random.uniform(1, max_amount_bet)
    if A_dollars >= B_dollars:
        f_decimal_l = B_dollars/A_dollars/m #favorite decimal line, adjusted for margin
        u_decimal_l = A_dollars/B_dollars/m #underdog decimal line, adjusted for margin
        l_f = 1/f_decimal_l #probability that favorite wins, based on favorite decimal line
        l_u = 1-1/u_decimal_l #probability that favorite wins, based on underdog decimal line
    else:
        f_decimal_l = A_dollars/B_dollars/m #favorite decimal line, adjusted for margin
        u_decimal_l = B_dollars/A_dollars/m #underdog decimal line, adjusted for margin
        l_f = 1/f_decimal_l #probability that favorite wins, based on favorite decimal line
        l_u = 1-1/u_decimal_l #probability that favorite wins, based on underdog decimal line
    return l_f, l_u, A, B, same_favorite #initial data needed to work with (betting lines and # of bets),
# # # # # # # # # # # # # # # # # # # # # here we generate it via uniform distributions

In [33]:
#calculate p,q,l odds for 1 betting scenario
def calculate_odds(l_f, l_u, A, B):
    m = 1+l_f-l_u
    q = l_f/m
    if A >= B:
        f_decimal_p = B/A #Underdog Betting Principle decimal odds for favorite
        p = 1/f_decimal_p #Unerdog Betting Principle hypothesized true odds for favorite
    else:
        f_decimal_p = A/B
        p = 1/f_decimal_p
    return p,q

In [34]:
def calculate_SF_and_position(p,q, same_favorite):
    if same_favorite:
        SF = p/q if p>=q else (1-p)/(1-q)
        position = 1 if p>=q else 0 #position is who we bet on, in case bet is placed
    else:
        SF = p/(1-q)
        position = 0 #always bet on underdog when true favorite is not bookie's favorite -> Underdog Betting Principle
    return SF, position

In [35]:
def play_bet(p, q, same_favorite, m, desired_SF_over_m_ratio):
    actual_SF_over_m_ratio, position = calculate_SF(p, q, same_favorite)/m
    if actual_SF_over_m_ratio >= desired_SF_over_m_ratio: #place bet
        expected_earnings_for_unit_bet = actual_SF_over_m_ratio
        possible_earnings_for_unit_bet = 1/(q*m) if position==1 else 1/((1-q)*m)
        outcome  = np.random.binomial(1, p) #this is our base assumption: reality follows the UBP hypothesized odds
        actual_earnings_for_unit_bet = possible_earnings_for_unit_bet if outcome==position else -1
    else:
        expected_earnings_for_unit_bet = 0
        actual_earnings_for_unit_bet = 0
    return actual_earnings_for_unit_bet, expected_earnings_for_unit_bet

In [37]:
#main
desired_SF_over_m_ratios = [1.05, 1.1, 1.2, 1.3, 1.5] #model controllable parameters, i.e. desired Safety 
m = 1.05 #5% bookie's margin, can keep constant without loss of gerality since we are setting SF/m ratio
Number_of_betting_scenarios = 10000
results = []
for i in range(len(desired_SF_over_m_ratios)):
    expected_earnings_for_each_scenario = []
    cumulative_expected_earnings = []
    actual_earnings_for_each_scenario = []
    cumulative_actual_earnings = []
    desired_SF_over_m_ratio = desired_SF_over_m_ratios[i]
    
    